In [3]:
# # download example data
# # !pip install gdown
# from downloaddata import download_example_data
# download_example_data()


In [1]:
%config IPCompleter.use_jedi = False



In [2]:
from wholeslidedata.iterators import create_batch_iterator
from wholeslidedata.image.wholeslideimagewriter import WholeSlideMaskWriter, HeatmapTileCallback, PredictionTileCallback
from wholeslidedata.image.wholeslideimage import WholeSlideImage
import time
from pprint import pprint
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from hooknet.configuration.config import create_hooknet
from pathlib import Path

In [3]:
user_config = './inference_configs/tlsgcinference.yml'
output_folder = Path('/home/user/output')
mode='training'

In [4]:
model = create_hooknet(user_config=user_config)

loading weights... /home/user/weights.h5


In [6]:

training_iterator = create_batch_iterator(mode=mode,
                                          user_config='./inference_configs/tlsgcinference.yml',
                                          presets=('slidingwindow',),
                                          cpus=4, 
                                          number_of_batches=-1, 
                                          return_info=True)

spacing =0.5
tile_size = 1024
output_size = 1030

image_path = None
wsm_writer = None
tls_heat_writer = None
for x_batch, y_batch, info in tqdm(training_iterator):
    x_batch = list(x_batch.transpose(1,0,2,3,4))
    predictions = model.predict_on_batch(x_batch, argmax=False)
    
    for idx, prediction in enumerate(predictions):
        
        sample_reference = info['sample_references'][idx]['reference']
        point = info['sample_references'][idx]['point']

        image = training_iterator.dataset.get_image_from_reference(sample_reference)
        if image_path is None or image.path != image_path:
            if image_path is not None:
                wsm_writer.save()
                tls_heat_writer.save()
                
            image_path = image.path
            wsm_writer = WholeSlideMaskWriter(callbacks=(PredictionTileCallback(),))
            tls_heat_writer = WholeSlideMaskWriter(callbacks=(HeatmapTileCallback(heatmap_index=1),))
            with WholeSlideImage(image_path) as wsi:
                shape = wsi.shapes[wsi.get_level_from_spacing(spacing)]
                real_spacing = wsi.get_real_spacing(spacing)
            wsm_writer.write(path=output_folder / (image_path.stem + '_hooknet.tif') , spacing=real_spacing, dimensions=shape, tile_shape=(tile_size,tile_size))
            tls_heat_writer.write(path=output_folder / (image_path.stem + '_hooknet_tls_heat.tif') , spacing=real_spacing, dimensions=shape, tile_shape=(tile_size,tile_size))
            
        c, r = point.x-output_size//4, point.y-output_size//4
        wsm_writer.write_tile(tile=prediction,coordinates=(int(c),int(r)), mask=y_batch[idx][0])
        tls_heat_writer.write_tile(tile=prediction,coordinates=(int(c),int(r)), mask=y_batch[idx][0])
    
wsm_writer.save()
tls_heat_writer.save()
training_iterator.stop()

  0%|          | 0/785 [00:00<?, ?it/s]

Creating: /home/user/output/20-1-2 H&E_hooknet.tif....
Spacing: 0.5034000148267036
Dimensions: (43823, 38744)
Tile_shape: (1024, 1024)
Creating: /home/user/output/20-1-2 H&E_hooknet_tls_heat.tif....
Spacing: 0.5034000148267036
Dimensions: (43823, 38744)
Tile_shape: (1024, 1024)
